In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_github = user_secrets.get_secret("github")
! rm -rf feedback2021
! git clone https://{secret_github}@github.com/VilmosProkaj/feedback2021.git
! pip install feedback2021/

Cloning into 'feedback2021'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 232 (delta 115), reused 173 (delta 56), pack-reused 0
Receiving objects: 100% (232/232), 61.74 KiB | 2.37 MiB/s, done.
Resolving deltas: 100% (115/115), done.
Processing ./feedback2021
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for feedback2021: filename=feedback2021-0.0.post1.dev33+g874c45e-py3-none-any.whl size=23927 

In [2]:
! pip install -U jax jaxlib dm_haiku optax
#!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# ! pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html  # Note: wheels only available on linux.
## !pip install --upgrade jax jaxlib==0.3.0+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html
#! pip install dm_haiku

     |████████████████████████████████| 912 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 65.4 MB 65.9 MB/s eta 0:00:01     |█████▍                          | 11.1 MB 63.7 MB/s eta 0:00:01
     |████████████████████████████████| 309 kB 71.0 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 73.0 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 6.4 MB/s  eta 0:00:01
  Created wheel for jax: filename=jax-0.3.1-py3-none-any.whl size=1054277 sha256=40e0ff19720c2c2ce64eff8fdf909565579ca90e6bd1ca05149bb4ef72106f02
  Stored in directory: /root/.cache/pip/wheels/04/14/e8/ee9de500f173ec900a5167686d9bb17c0171ed678680b96a57
Successfully built jax
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.70
    Uninstalling jaxlib-0.1.70:

In [3]:
import kaggle_init

In [4]:
kaggle_init.on_kaggle(), kaggle_init.is_cuda_available(), kaggle_init.is_tpu_available()

(True, False, True)

In [5]:
# %env XLA_PYTHON_CLIENT_MEM_FRACTION=.75
compute_on_tpu = True
if compute_on_tpu:
    if kaggle_init.is_tpu_available():
        from feedback2021.jax_tpu_init import jax_tpu_init
        jax_tpu_init()       
    else:
        import os
        os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=2'
#import jax
#jax.devices()

In [ ]:
# jax_tpu_init??

In [ ]:
LOG_TO_WANDB = False

In [ ]:
if LOG_TO_WANDB:
    !pip install --upgrade wandb -q # experiment tracking

In [ ]:
if LOG_TO_WANDB:
    import wandb
    import os
    os.environ["WANDB_PROJECT"] = "kaggle_feedback"
    os.environ["WANDB_ENTITY"] = "prvi"
    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "gradient"

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        api_key = user_secrets.get_secret("wandb")
        os.environ["WANDB_API_KEY"] = api_key
        wandb.login()
        wandb.init(dir="/tmp/") 
    except:
        print('If you want to use your W&B account, '
              'go to Add-ons -> Secrets and provide your W&B access token.\n'
              'Use the Label name `wandb`. \n'
              'Get your W&B access token from here: https://wandb.ai/authorize')

In [6]:
from feedback2021.helper import id2label, label2id

from feedback2021.prepare_data import (create_train_dataset_pd, 
                                       to_chunk_data, 
                                       chunk_mapping,
                                       add_input_ids, 
                                       add_labels,
                                       add_rle_word2,
                                       has_name,
                                    )

import feedback2021.metric as metric

from feedback2021.postprocess import (mk_metric, 
                                      mk_prediction_transform, 
                                      mk_binary_metric, 
                                      mk_binary_prediction_transform)

from feedback2021.visualize import show_result

import feedback2021.hk_roberta as hk_roberta

In [7]:
# CONFIG
just_test = False
experiment_id = 1
task = "token_classification"
model_checkpoint = "distilroberta-base"
# "roberta-base" 
# "allenai/longformer-base-4096" 
# "distilroberta-base" # "microsoft/deberta-v3-xsmall" #"roberta-base"
if just_test:
    max_length = 128
    stride = 64
else:
    max_length = 512
    stride = 128
max_rle_length = 32
min_tokens = 6
model_path = f'{model_checkpoint.split("/")[-1]}-{experiment_id}'
data_from_wandb = False
save_to_wandb = False or not data_from_wandb


## Data Preprocessing

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#import tensorflow.config
#tensorflow.config.experimental.set_visible_devices([], "GPU")

In [9]:
from feedback2021.clean_train_data import mk_clean_train_data
if just_test:
    cleaned_train = mk_clean_train_data(num_records=100) #cleaned_train[:100]
else:
    cleaned_train = mk_clean_train_data()

text_id2idx = dict(zip(list(cleaned_train.index.values), range(len(cleaned_train))))
idx2text_id = {v:k for k,v in text_id2idx.items()}
cleaned_train.index = range(len(cleaned_train))

data = create_train_dataset_pd(cleaned_train_df=cleaned_train, 
                               tokenizer=tokenizer, 
                               fields = ['rle_token'],
                               verbose=True)

from sklearn.model_selection import train_test_split

data = dict(zip(['train','test'], 
                train_test_split(data,
                                 test_size=0.1, 
                                 shuffle=True, 
                                 random_state=42)))


2022-03-11 15:02:25.972154: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-11 15:02:30.007837: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-11 15:02:30.007900: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)


reformat train data:   0%|          | 0/144293 [00:00<?, ?it/s]

reading essays:   0%|          | 0/15594 [00:00<?, ?it/s]

searching for disaligned labels:   0%|          | 0/15594 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (723 > 512). Running this sequence through the model will result in indexing errors


adding token rle:   0%|          | 0/15594 [00:00<?, ?it/s]

In [14]:
data['train'].head(), data['test'].head()

(                                                 text  \
 18  Dear Principal,\n\nI think that making someone...   
 30  Dear Principal,\n\nI believe that the best pol...   
 73  Have you ever been bored in a class room such ...   
 33  The idea of having cars that can do the drivin...   
 90  Hey you! Do you need some advice about somethi...   
 
                                              rle_char  \
 18  [(2, 17, 100), (4, 102, 245), (3, 512, 611), (...   
 30  [(2, 17, 74), (3, 77, 354), (6, 357, 486), (7,...   
 73  [(1, 0, 328), (2, 330, 473), (4, 476, 540), (3...   
 33  [(1, 0, 396), (2, 398, 565), (3, 568, 1688), (...   
 90  [(1, 0, 155), (2, 157, 284), (4, 293, 312), (4...   
 
                                             input_ids  \
 18  [23314, 13619, 6, 50118, 50118, 100, 206, 14, ...   
 30  [23314, 13619, 6, 50118, 50118, 100, 679, 14, ...   
 73  [17781, 47, 655, 57, 23809, 11, 10, 1380, 929,...   
 33  [133, 1114, 9, 519, 1677, 14, 64, 109, 5, 1428...   
 90  [1336

In [ ]:
if LOG_TO_WANDB and save_to_wandb:
    data.remove_columns([#'input_ids', 
                         #'rle_token', 
                         'labels', 
                         #'offset_mapping'
    ]).save_to_disk('data')

    artifact = wandb.Artifact('data', description='train test split', type='dataset')
    artifact.add_dir('data')
    wandb.log_artifact(artifact)
    !ls -sRh data
    

In [ ]:
# if  not isinstance(data.column_names, dict):
#     data = data.train_test_split(test_size=0.1, shuffle=True, seed=42)
    
# chunk_data = to_chunk_data(data, 
#                            chunk_len=max_length, 
#                            stride=stride, 
#                            prefix=[tokenizer.bos_token_id],
#                            postfix=[tokenizer.eos_token_id])
# chunk_data

In [10]:
import itertools

def chunk_mapping(chunk_len=512,
                  stride=256,
                  rle_len=None,
                  prefix=[0], 
                  postfix=[2]):
    extra_len = len(prefix)+len(postfix)
    chunk_len = chunk_len-extra_len
    
    def f(examples, idx=None):
        res = dict(input_ids=[], rle_token=[], offset=[], text_id=[])
        for ids, token_rle, xid in zip(examples['input_ids'], 
                                    examples.get('rle_token', itertools.repeat([])), 
                                    examples['id']):
            linput = len(ids)
            offset = 0
            while linput > offset:
                offset = max(0, min(offset, linput-chunk_len))
                end_pos = min(linput, offset+chunk_len)
                s = slice(offset, end_pos)
                res['input_ids'].append(prefix+ids[s]+postfix)
                res['text_id'].append(xid)
                res['offset'].append(offset)
                rle = [type(rle)((rle[0], 
                                  max(rle[1]-offset,0) + len(prefix), 
                                  min(rle[2], end_pos) - offset + len(prefix)))
                       for rle in token_rle if rle[2]>offset and rle[1]<end_pos]
                if rle_len is not None:
                    rle=rle[:rle_len]
                res['rle_token'].append(rle)
                offset = offset+stride if end_pos<linput else linput

        if not any(res['rle_token']):
            res.pop('rle_token')
        return res
    return f

def chop_up(data, tokenizer, max_length, stride, max_rle_len=None):
    examples = {k: list(data[k]) for k in data.columns}
    examples['id'] = map(int, data.index.values)
    f = chunk_mapping(chunk_len=max_length, 
                      stride=stride, 
                      rle_len=max_rle_len,
                      prefix=[tokenizer.bos_token_id],
                      postfix=[tokenizer.eos_token_id])
    return f(examples)

import numpy as np

def mk_collate_fn(tokenizer, max_length=512, max_rle_length=32):
    def collate_fn(features):
        
        batch = features.copy()
        p = tokenizer.pad_token_id
        batch['input_ids'] = np.array([f+[p]*(max_length-len(f)) for f in batch['input_ids']],
                                      dtype=np.int32)
        batch['rle_token'] = np.array([f+[(0,-1,-1)]*(max_rle_length-len(f)) for f in batch['rle_token']], 
                                      dtype=np.int32)
        return batch
    
    return collate_fn

In [ ]:
# def chop_up(data, tokenizer, max_length, stride):
#     examples = {k: list(data[k]) for k in data.columns}
#     examples['labels'] = list(data['labels'])
#     examples['id'] = list(data.index.values)
#     f = chunk_mapping(chunk_len=max_length, 
#                       stride=stride, 
#                       prefix=[tokenizer.bos_token_id],
#                       postfix=[tokenizer.eos_token_id])
#     return f(examples)

# def to_records(data):
#     return [dict(zip(data.keys(), rec))  for rec in zip(*data.values())]
    

In [11]:
chunk_data = {
    k: chop_up(v, tokenizer, max_length=max_length, stride=stride, max_rle_len=32)
    for k,v in data.items()
}

In [12]:
import yaml
print(yaml.dump(
    {
        k: {
            k0: f'size={len(v0[0])}, type={type(v0[0][0]).__name__}' 
            if isinstance(v0[0], list) else v0[0] 
            for k0, v0 in v.items()
        } 
     for k,v in chunk_data.items()
    }
))

test:
  input_ids: size=216, type=int
  offset: 0
  rle_token: size=9, type=Block
  text_id: 10360
train:
  input_ids: size=512, type=int
  offset: 0
  rle_token: size=8, type=Block
  text_id: 7969



In [64]:
(data['train'].loc(0)[18]['rle_token'],
 data['train'].loc(0)[18].index.values, #['text_id'][1],
 chunk_data['train']['rle_token'][:4], 
 chunk_data['train']['offset'][:4],
 chunk_data['train']['text_id'][:4],
)

([Position(5:21),
  Claim(22:50),
  Evidence(105:129),
  Claim(132:152),
  Evidence(153:186),
  Claim(187:199),
  Evidence(200:252),
  Concluding Statement(282:383)],
 array(['text', 'rle_char', 'input_ids', 'offset_mapping', 'rle_token'],
       dtype=object),
 [[Position(6:22), Claim(23:51), Evidence(106:127)],
  [Evidence(42:66), Claim(69:89), Evidence(90:123), Claim(124:127)],
  [Evidence(1:2),
   Claim(5:25),
   Evidence(26:59),
   Claim(60:72),
   Evidence(73:125)],
  [Claim(1:8), Evidence(9:61), Concluding Statement(91:127)]],
 [0, 64, 128, 192],
 [18, 18, 18, 18])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# if not has_name(data, 'rle_word'):
#     from feedback2021.helper import Block
#     def add_rle_word2(data):
#         # assert has_name(data, 'offset_mapping'), 'add input_ids first!'
#         if not has_name(data,'word_mapping'):
#             data = add_word_mapping(data)
#         return data.map(lambda x: {'rle_word': [Block(t).inv_map(x['word_mapping']) 
#                                                  for t in x['rle_char']]},
#                             desc='rle to word coordinates')
#     data = add_rle_word2(data)

In [ ]:
# from collections import defaultdict
# import numpy as np
# word_counts = defaultdict(list)
# from  datasets import concatenate_datasets
# all_data = concatenate_datasets(list(data.values()))
# for x in all_data['rle_word']:
#     for cls_id, start, end  in x:
#         word_counts[id2label[cls_id]].append(end-start)

# for k, v in word_counts.items():
#     plt.hist(v,bins=np.arange(1,max(v)+1)-0.5)
#     plt.title(f'{k} min:{min(v)} max: {max(v)}, 2%,5%: {np.percentile(v,[2,5])}')
#     plt.show()

## Model and Training

In [ ]:
# chunk_data = chunk_data.rename_column('labels','label')
# DataLoader?

In [ ]:
# import numpy as np
# def mk_collate_fn(tokenizer, max_length=512):
#     def collate_fn(features):
        
#         batch ={key: np.array([f[key]+[t]*(max_length-len(f[key])) for f in features], dtype=np.int32)
#                 for t,key in zip([tokenizer.pad_token_id,-100],['input_ids','labels'])}
#         return batch
#     return collate_fn
# from torch.utils.data import DataLoader

# train_dataset = DataLoader(chunk_data_list, shuffle=True, #.remove_columns(['offset', 'text_id']), 
#                            batch_size=16,
#                            collate_fn=mk_collate_fn(tokenizer, 512))

In [13]:
%%time
import tensorflow.data as tfdata
#ds=
# import numpy as np

# def mk_collate_fn(tokenizer, max_length=512):
#     def collate_fn(features):
        
#         batch = features.copy()
#         for t, key in zip([tokenizer.pad_token_id,-100], ['input_ids','labels']):
#             batch[key] = np.array([f+[t]*(max_length-len(f)) for f in batch[key]], 
#                                   dtype=np.int32)
#         return batch
    
#     return collate_fn

    
collate_fn = mk_collate_fn(tokenizer, max_length, max_rle_length=32)

ds ={k: tfdata.Dataset.from_tensor_slices(collate_fn(v)) for k,v in chunk_data.items()}
#(chunk_data_list)

CPU times: user 7.3 s, sys: 133 ms, total: 7.43 s
Wall time: 7.41 s


In [14]:
ds['train']

<TensorSliceDataset shapes: {input_ids: (512,), rle_token: (32, 3), offset: (), text_id: ()}, types: {input_ids: tf.int32, rle_token: tf.int32, offset: tf.int32, text_id: tf.int32}>

In [15]:
batch_size = 16 if kaggle_init.is_tpu_available() else 2
dummy_data = tfdata.Dataset.from_tensor_slices(
    {
        'input_ids': np.zeros((batch_size, max_length), dtype=np.int32),
        'rle_token': np.array([[(0,-1,-1)]*max_rle_length]*batch_size, dtype=np.int32),
        # 'labels': -100*np.ones((batch_size, max_length), dtype=np.int32),
        'text_id': -1*np.ones(batch_size, dtype=np.int32),
        ## np.array([-1]*batch_size), #np.array(['0'*11]*batch_size),
        'offset': np.zeros(batch_size, dtype=np.int32)
    }
)

In [16]:
dummy_data

<TensorSliceDataset shapes: {input_ids: (512,), rle_token: (32, 3), text_id: (), offset: ()}, types: {input_ids: tf.int32, rle_token: tf.int32, text_id: tf.int32, offset: tf.int32}>

In [17]:
import jax
jax.devices(), jax.device_count()

([TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
  TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
  TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
  TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
  TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
  TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
  TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
  TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)],
 8)

In [18]:
def subset(x):
    return {k: x[k] for k in ['input_ids', 'labels', 'offset']}

train_dataset = (ds['train'].
                 # map(subset).
                 repeat().
                 shuffle(4096).
                 batch(batch_size=batch_size).
                 batch(batch_size=jax.device_count()).
                 prefetch(tfdata.experimental.AUTOTUNE).
                 as_numpy_iterator())

test_dataset = (ds['test'].
                concatenate(dummy_data.repeat(jax.device_count())).
                #map(subset).
                batch(batch_size=batch_size, drop_remainder=True).
                batch(batch_size=jax.device_count(), drop_remainder=True).
                prefetch(tfdata.experimental.AUTOTUNE)
)

In [19]:
len(ds['test']), len(ds['train'])

(2904, 25681)

In [ ]:
class Metric:
    def __init__(self):
        self.reset()
    def reset(self):
        self._value = 0
        self._n = 0
    def update(self,v):
        self._n += 1
        self._value += v
    def value(self):
        return self._value/self._n

In [41]:
{k: f'shape={v.shape}, dtype={v.dtype}' for k,v in next(iter(train_dataset)).items()}

{'input_ids': 'shape=(2, 2, 128), dtype=int32',
 'rle_token': 'shape=(2, 2, 32, 3), dtype=int32',
 'offset': 'shape=(2, 2), dtype=int32',
 'text_id': 'shape=(2, 2), dtype=int32'}

In [ ]:
local_device_count = jax.device_count()
local_device_count

In [102]:
# jnp.broadcast_to(jnp.array([[1,2],[3,4]]), (3,2,2))

DeviceArray([[[1, 2],
              [3, 4]],

             [[1, 2],
              [3, 4]],

             [[1, 2],
              [3, 4]]], dtype=int32)

In [28]:
from feedback2021.hk_roberta import *

class DecoderLayer(hk.Module):
    def __init__(self, config, name=None):
        super().__init__(name=name)
        self._config = config 
    
    def __call__(self, features, hidden, mask, is_training):
        config = self._config
        
        init_scale = 2. / config['num_hidden_layers']
        attn_dropout_rate = config['attention_probs_dropout_prob'] if is_training else 0.
        hidden_dropout_rate = config['hidden_dropout_prob'] if is_training else 0.
        ln_kwargs = {} if 'layer_norm_eps' not in config else {'eps': config['layer_norm_eps']}
        
        
        cross_attn = hk.MultiHeadAttention(
                num_heads=config['num_attention_heads'],
                key_size=config['hidden_size']//config['num_attention_heads'],
                w_init_scale=init_scale,
                name='cross_attention')(hidden, features, features, mask=mask)

        cross_attn = hk.dropout(hk.next_rng_key(), attn_dropout_rate, cross_attn)
        hidden = hidden + cross_attn
        hidden = layer_norm(hidden, name='cross_attention_layernorm', **ln_kwargs)
        
        h_attn = hk.MultiHeadAttention(
                num_heads = config['num_attention_heads'],
                key_size  = config['hidden_size']//config['num_attention_heads'],
                w_init_scale = init_scale,
                name='self_attention')(hidden, hidden, hidden, mask=None)

        h_attn = hk.dropout(hk.next_rng_key(), attn_dropout_rate, h_attn)
        hidden = hidden + h_attn
        hidden = layer_norm(hidden, name='self_attention_layernorm', **ln_kwargs)
        
        
        h_dense = DenseBlock(config, name='mlp')(hidden)
        
        h_dense = hk.dropout(hk.next_rng_key(), hidden_dropout_rate, h_dense)
        hidden = hidden + h_dense
        hidden = layer_norm(hidden, name='layernorm', **ln_kwargs)
        
        return hidden

class Decoder(hk.Module):
    """A transformer stack."""

    def __init__(self, config, name=None):
        super().__init__(name=name)
        self._config = config 
        
    def __call__(self, 
                 features: jnp.ndarray,
                 mask: Optional[jnp.ndarray],
                 is_training: bool) -> jnp.ndarray:
        """Connects the transformer.
        Args:
          h: Inputs, [B, T, H].
          mask: Padding mask, [B, T].
          is_training: Whether we're training or not.
        Returns:
          Array of shape [B, T, H].
        """
        config = self._config
        
        hidden_size = config['hidden_size']
        num_token = config['num_token']
        init_scale = config['initializer_range']
        
        if mask is not None:
            mask = mask[:, None, None, :]

        hidden = hk.get_parameter('hidden', 
                                  [num_token, hidden_size], 
                                  init=hk.initializers.TruncatedNormal(init_scale))
        
        hidden = jnp.broadcast_to(hidden, features.shape[:1]+hidden.shape)
        
        for i in range(config['num_hidden_layers']):
            hidden = DecoderLayer(config, name=f'layer_{i}')(features, hidden, mask, is_training=is_training)
                             
        return hidden

class FinalAttention(hk.Module):
    """
        It takes te hidden vectors and the feature vectors and computes for each hidden vector 
        two probability distribution over the feature vector positions.
        # Multi-headed attention mechanism.

        # As described in the vanilla Transformer paper:
        # "Attention is all you need" https://arxiv.org/abs/1706.03762
    """

    def __init__(
          self,
          config,
          name=None,
    ):
    #       num_heads: int,
    #       key_size: int,
    #       w_init_scale: float,
    #       value_size: Optional[int] = None,
    #       model_size: Optional[int] = None,
    #       name: Optional[str] = None,
    # ):
    #     
        super().__init__(name=name)
        self._config = config
        self.num_heads = config['num_attention_heads']
        self.key_size = config['key_size']
        # self.value_size = value_size or key_size
        # self.model_size = model_size or key_size * num_heads
        self.w_init = hk.initializers.VarianceScaling(config['initializer_range'])

    def __call__(
          self,
          features: jnp.ndarray,
          hidden: jnp.ndarray,
          # value: jnp.ndarray,
          mask: Optional[jnp.ndarray] = None,
    ) -> jnp.ndarray:
        """Compute (optionally masked) MHA with queries, keys & values."""
        
        config = self._config
        
        query_heads = self._linear_projection(features, "query")
        key_heads = self._linear_projection(hidden, "key")
        #value_heads = self._linear_projection(value, self.value_size, "value")

        attn_logits = jnp.einsum("...thd,...Thd->...tTh", query_heads, key_heads)
        sqrt_key_size = np.sqrt(self.key_size).astype(hidden.dtype)
        attn_logits = attn_logits / sqrt_key_size
        if mask is not None:
            mask = mask[:, :,  None, None]
            assert len(mask.shape) == len(attn_logits.shape)
            attn_logits = jnp.where(mask, attn_logits, -1e30)

        return attn_logits
        # attn_weights = jax.nn.softmax(attn_logits)
        # attn = jnp.einsum("...htT,...Thd->...thd", attn_weights, value_heads)
        # Concatenate attention matrix of all heads into a single vector.
        # attn_vec = jnp.reshape(attn, (*query.shape[:-1], -1))

        # return hk.Linear(self.model_size, w_init=self.w_init)(attn_vec)

    @hk.transparent
    def _linear_projection(
          self,
          x: jnp.ndarray,
          # head_size: int,
          name: Optional[str] = None
    ) -> jnp.ndarray:
        y = hk.Linear(self.num_heads * self.key_size, w_init=self.w_init, name=name)(x)
        return y.reshape((*x.shape[:-1], self.num_heads, self.key_size))

# class FinalAttention(hk.Module):
#     """A transformer stack."""

#     def __init__(self, config, name=None):
#         super().__init__(name=name)
#         self._config = config 

#     def __call__(self, features, hidden, mask=None, is_training=False):
#         config = self._config
#         init_scale = config['initializer_range']
        
#         if mask is not None:
#             mask = mask[:, :,  None]

        
#         output = hk.MultiHeadAttention(
#                 num_heads=config['num_attention_heads'],
#                 key_size=config['hidden_size']//config['num_attention_heads'],
#                 model_size=config['output_size'],
#                 w_init_scale=init_scale,
#                 name='final_attention')(features, hidden, hidden, mask=None)

#         if mask is not None:
#             assert len(mask.shape) == len(output.shape), "mask shape=%s output shape= %s" % (mask.shape,output.shape)
#             output = jnp.where(mask, output, -1e30)
            
#         return output
    
def check_config(config):
    return config
    
def mk_decoder_fn(config):
    
    config = check_config(config)
    
    def fn(features, mask, is_training=True):
        
        #input_embeddings, input_mask = Embedding(config)(data, is_training)
        
        
        decoder = Decoder(config['decoder'], name='decoder')
        hidden = decoder(features, mask, is_training=is_training)
        final_attn = FinalAttention(config['final_attn'], name='final_attn')
        pos_logits = final_attn(features, hidden, mask) #, is_training=is_training)
        logits = hk.Linear(len(config['id2label']))(hidden)
        #print(pos_logits.shape, logits.shape, hidden.shape)
        return pos_logits, logits
    
    return fn

def mk_feature_fn(config):

    config = check_config(config)
    
    def fn(data, is_training=True):
        
        embeddings, mask = Embedding(config)(data, is_training)
        transformer = Transformer(config, name='encoder')
        features = transformer(embeddings, mask, is_training)
        #print(features.shape, mask.shape)
        return features, mask
    
    return fn
    
def mk_loss_fn(config):
    
    def loss_fn(pos_logits, logits, data):
    #                params,
    #                rng,
    #                data: Mapping[str, jnp.ndarray],
    #                is_training: bool = True) -> jnp.ndarray:
        """Compute the loss on data wrt params."""


    #     logits = forward_fn(params, rng, data, is_training)
        cls_target = data['rle_token'][...,0]
        pos_target = data['rle_token'][...,1:]
        
        cls_onehot = jax.nn.one_hot(cls_target, len(config['id2label']))
        assert logits.shape == cls_onehot.shape, "shape mismatch: %s !=  %s" %(logits.shape, 
                                                                               cls_onehot.shape)
        
        cls_loss = -jnp.mean(cls_onehot * jax.nn.log_softmax(logits))
        # cls_loss = cls_loss/jnp.clip(jnp.sum(mask), 1)

        mask = jnp.greater(cls_target, 0)

        
        # pos_logits shape: batch_size x seq_len x num_pred x 2
        # pos_target shape: batch_size x num_pred x 2
        # mask shape: batch_size x num_pred 
        # pos_mask.shape = batch_size x seq_len

        pos = jnp.arange(pos_logits.shape[-3])[:, None, None]
        pos_target = pos_target[:, None, :, :]
        weights = 1/(jnp.diff(pos_target, axis=-1)+1)
        dist = jnp.abs(pos-pos_target)*weights
        
        mask = mask[:,None,:,None]
        pos_mask = jnp.greater(data['input_ids'], max(config['pad_token_id'], 
                                                      config['bos_token_id'],
                                                      config['eos_token_id']))
        # print(pos_mask.shape, mask.shape, pos_logits.shape)
        pos_mask = pos_mask[:, :, None, None]*mask
        
        pos_loss = -jnp.mean(pos_mask*dist*jax.nn.log_softmax(pos_logits, axis=-3))
        pos_loss = pos_loss/jnp.clip(jnp.mean(pos_mask),1e-8)

        return cls_loss+pos_loss
    
    return loss_fn

    
def mk_train_forward_fn(backbone, net, loss_fn):
    
    def fn(params, backbone_params, rng, data, is_training=True):
        rng1, rng2 = jax.random.split(rng)
        feature, mask = backbone.apply(backbone_params, rng1, data, is_training=is_training)
        pos_logits, logits = net.apply(params, rng2, feature, mask, is_training=is_training)
        return loss_fn(pos_logits, logits, data)
        
    return fn

class Updater2:
    """A stateless abstraction around an init_fn/update_fn pair.
    This extracts some common boilerplate from the training loop.
    """

    def __init__(self, 
                 backbone,
                 net,
                 loss_fn,
                 optimizer: optax.GradientTransformation):
        
        self._net_init = net.init
        self.backbone = backbone
        self._loss_fn = mk_train_forward_fn(backbone, net, loss_fn)
        # lambda params,rng, data: loss_fn(net.apply(params, rng, data, is_training=True), data)
        self._opt = optimizer

    @functools.partial(jax.jit, static_argnums=0)
    def init(self, rng, data, pretrained_params):
        """Initializes state of the updater."""
        out_rng, init_rng = jax.random.split(rng)
        features, mask = self.backbone.apply(pretrained_params, init_rng, data)
        params = self._net_init(init_rng, features, mask)
#         if pretrained_params is not None:
#             params = hk.data_structures.merge(params, pretrained_params)
#         #params = hk.data_structures.map(lambda x: jnp.stack([x]*local_device_count), params)
        opt_state = self._opt.init(params)
        # rng = jax.random.PRNGKey(FLAGS.train_init_random_seed)
        #rng = jnp.broadcast_to(rng, (local_device_count,) + rng.shape)
        out = dict(
            step = np.array(0),
            rng = out_rng,
            opt_state = opt_state,
            params = params,
            backbone_params = pretrained_params,
            loss = np.array(0),
        )
        return out

    @functools.partial(jax.jit, static_argnums=0)
    def update(self, state: Mapping[str, Any], data: Mapping[str, jnp.ndarray]):
        """Updates the state using some data and returns metrics."""
        rng, new_rng = jax.random.split(state['rng'])
        params = state['params']
        backbone_params = state['backbone_params']
        loss, grads = jax.value_and_grad(self._loss_fn)(params, backbone_params, rng, data)

        if compute_on_tpu:
            grads = jax.lax.pmean(grads, 'i')


        updates, opt_state = self._opt.update(grads, state['opt_state'])
        params = optax.apply_updates(params, updates)

        new_state = {
            'step': state['step'] + 1,
            'rng': new_rng,
            'opt_state': opt_state,
            'params': params,
            'backbone_params': backbone_params,
            'loss': state['loss'] + loss
        }

        return new_state 


In [21]:
def mk_net_eval2(net, backbone):
    def run_net(params, backbone_params, rng, data):
        rng1, rng2 = jax.random.split(rng)
        feature, mask = backbone.apply(backbone_params, rng1, data, is_training=False)
        pos_logits, logits = net.apply(params, rng2, feature, mask, is_training=False)
        return pos_logits, logits
    
    run_net = jax.pmap(run_net, axis_name='i')
    return run_net

def predict2(net, state, data):
    predictions = []
    data = data.map(lambda x: ({'input_ids':x['input_ids']},
                               x['input_ids'],
                               x['text_id'], 
                               x['offset'])).as_numpy_iterator()
    
    params = state['params']
    backbone_params=state['backbone_params']
    rng = state['rng']
    
    for data, ids, text_id, offset  in data:   
        # feature, mask = backbone.apply(backbone_params, rng1, data, is_training=False)
        pos_logits, logits = net(params, backbone_params, rng, data) 
        #feature, mask, is_training=False)
        pos = jnp.arange(pos_logits.shape[-3])[:, None, None]
        pos = jnp.round(jnp.sum(jax.nn.softmax(pos_logits, axis=-3)*pos, axis=-3))
        cls_pred = jax.device_get(jnp.argmax(logits, axis=-1))
        cls_pred = cls_pred.reshape((-1,)+cls_pred.shape[2:])
        del logits
        pos0 = jax.device_get(pos)
        pos0 = pos0.reshape((-1,)+pos0.shape[2:])
        del pos
        preds = [[tuple((cls_id, start, end)) for cls_id,(start,end) in zip(cls_ids,pos) if cls_id>0]
                 for cls_ids, pos in zip(cls_pred, pos0)]
        ## logits0 = logits0.reshape((-1,)+logits0.shape[2:])
        ## 0,1,2 are <s> </s> <pad>!
        # mask = (ids>2).reshape((-1,)+ids.shape[2:])
        predictions.extend(zip(text_id.reshape(-1), 
                               offset.reshape(-1), 
                               preds))
                               # (logit[m] for logit,m in zip(logits0, mask))))
                               # data['input_ids'].reshape((-1,)+data['input_ids']))
    return predictions

from collections import defaultdict


def collect_chunks2(raw_predictions):
    predictions = defaultdict(set)
    for idx, offset, pred in raw_predictions:
        predictions[idx].update([(cls_id, start-1+offset, end-1+offset) 
                                 for cls_id, start, end in pred])
    return dict(predictions)

# def combine_offset_list(offset_lst):
#     offset_lst = [(offset, np.asarray(c)) for offset, c in offset_lst]
#     total_len = max(offset+len(c) for offset, c in offset_lst)
#     chunk_shape = offset_lst[0][1].shape
#     res = np.zeros((total_len,)+chunk_shape[1:])
#     cnt = np.zeros((total_len,)+(1,)*(len(chunk_shape)-1))
#     for i, (o, c) in enumerate(offset_lst):
#         res[o:o+len(c)] += c
#         cnt[o:o+len(c)] += 1
#     return res/cnt.clip(1)

from feedback2021.helper import label2id, id2label, Block
# def pred_rle(pred):
    
#     res = []
#     cur_cls = -1
#     min_cls = label2id.get('None',-1)
    
#     for i, cls_id in enumerate(pred):
        
#         if (cur_cls>min_cls) and (cur_cls!=cls_id):
#             res.append(Block((cur_cls, start, i)))
#             cur_cls = -1
        
#         if cls_id>min_cls and cur_cls==-1: 
#             cur_cls, start = cls_id, i
    
#     if cur_cls>min_cls:
#         res.append(Block((cur_cls, start, i+1)))
    
#     return res


import  feedback2021.metric as metric

def compute_metric2(eval_net, state, chunked_data, raw_df):
    predictions = predict2(eval_net, state, chunked_data)
    predictions = collect_chunks2(predictions)
#     predictions = {k: pred_rle(np.argmax(combine_offset_list(v), axis=-1)) 
#                    for k,v in predictions.items()}
    
    raw_data = zip(raw_df.index, raw_df['rle_token']) #offset_mapping'], raw_df['word_mapping'])
#     predictions = [
#             [b.map(om).inv_map(wm) for b in predictions[idx]]
#             for idx, om, wm in raw_data
#         ]

    scores = np.array([ 
            metric.score_example(y_pred=predictions[idx], y_true=y_true)
            for idx,y_true in raw_data #zip(predictions, raw_df['rle_word'])
        ]).sum(axis=0)
        
    res = {id2label[i+1]: metric.f1(*score) for i, score in enumerate(scores)}
    res['f1'] = sum(res.values())/len(res)
        
    return res
    
    
    

## predict(jax.pmap(net.apply, axis_name='i'), state, dataset)

        
# def get_predictions(net, loss_fn, state, test_ds):

#     def run_net(state, data):
#         logits = net.apply(state['params'], state['rng'], data, is_training=False)
#         loss = loss_fn(logits, data)
#         pred = jax.numpy.argmax(logits, axis=-1)
#         state['loss'] = state['loss'] + loss
#         return pred, state
        
    
#     run_net = jax.pmap(run_net, axis_name='i')
#     predictions = []
#     for batch in test_ds.as_numpy_iterator():
#         pred, state = run_net(state, batch)
#         predictions.append(pred)
        
#     predictions = [ pred 
#                     for preds in jax.device_get(predictions) 
#                     for pred in preds.reshape((-1,) + preds.shape[2:]) ]
#     return predictions, state

# def eval_net(net, loss_fn, state, test_ds):

#     def run_net(state, data, loss, cnt):
#         cnt0 = jnp.greater(data['labels'].max(axis=-1),0).sum()
#         logits = net.apply(state['params'], state['rng'], data, is_training=False)
#         loss0 = loss_fn(logits, data) * data['labels'].shape[0]
#         #pred = jax.numpy.argmax(logits, axis=-1)
#         #state['loss'] = state['loss'] + loss
#         return loss + loss0, cnt + cnt0
        
    
#     run_net = jax.pmap(run_net, axis_name='i')
#     #predictions = []
#     loss = np.zeros(jax.device_count())
#     cnt = np.zeros(jax.device_count())
#     for batch in test_ds.as_numpy_iterator():
#         loss, cnt = run_net(state, batch, loss, cnt)
#         # predictions.append(pred)
#     return float(loss.sum()/cnt.sum())

In [ ]:
config=dict(
    decoder = dict(
        num_token = max_rle_length,
        hidden_size = 64*12,
        initializer_range=0.02,
        hidden_dropout_prob = 0.1,
        attention_probs_dropout_prob=0.1,
        #layer_norm_eps = 1e-5,
        intermediate_size = 4*64*12
        hidden_act = 'gelu',
        num_hidden_layers = 12, # 6,12,24,32?
        num_attention_heads = 12,
        ),
    final_attn = dict(
        key_size = 64, #16,#64*12,
        initializer_range=0.02,
        # hidden_dropout_prob = 0.1,
        # attention_probs_dropout_prob=0.1,
        # layer_norm_eps = 1e-5,
        # intermediate_size = 64, # 4*64*12
        # hidden_act = 'gelu',
        # num_hidden_layers = 3, # 6,12,24,32?
        num_attention_heads = 2,
        output_size = 2
    ),
    pad_token_id =1,
    vocab_size = 10, #52000
    max_position_embeddings = 32, # 514
    
    hidden_size = 6*16,#64*12,
    initializer_range=0.02,
    hidden_dropout_prob = 0.1,
    attention_probs_dropout_prob=0.1,
    #layer_norm_eps = 1e-5,
    intermediate_size = 64, # 4*64*12
    hidden_act = 'gelu',
    num_hidden_layers = 2, # 6,12,24,32?
    num_attention_heads = 6,
    id2label = id2label,
)
config

In [22]:
import haiku as hk
import jax.numpy as jnp
import optax
import functools
from typing import Any, Mapping

def mk_loss_fn(config):
    
    def loss_fn(logits, data):
    #                params,
    #                rng,
    #                data: Mapping[str, jnp.ndarray],
    #                is_training: bool = True) -> jnp.ndarray:
        """Compute the loss on data wrt params."""


    #     logits = forward_fn(params, rng, data, is_training)
        targets = jax.nn.one_hot(data['labels'], len(config['id2label']))
        assert logits.shape == targets.shape

        mask = jnp.not_equal(data['input_ids'], config['pad_token_id'])
        mask = mask * jnp.greater_equal(data['labels'], 0)
        loss = -jnp.sum(targets * jax.nn.log_softmax(logits), axis=-1)
        loss = jnp.sum(loss * mask, axis=-1) / jnp.clip(jnp.sum(mask, axis=-1),1)

        return jnp.mean(loss)
    
    return loss_fn

class Updater:
    """A stateless abstraction around an init_fn/update_fn pair.
    This extracts some common boilerplate from the training loop.
    """

    def __init__(self, 
                 net, 
                 loss_fn,
                 optimizer: optax.GradientTransformation):
        
        self._net_init = net.init
        self._loss_fn = lambda params,rng, data: loss_fn(net.apply(params, rng, data, is_training=True), data)
        self._opt = optimizer

    @functools.partial(jax.jit, static_argnums=0)
    def init(self, rng, data, pretrained_params=None):
        """Initializes state of the updater."""
        out_rng, init_rng = jax.random.split(rng)
        params = self._net_init(init_rng, data)
        if pretrained_params is not None:
            params = hk.data_structures.merge(params, pretrained_params)
        #params = hk.data_structures.map(lambda x: jnp.stack([x]*local_device_count), params)
        opt_state = self._opt.init(params)
        # rng = jax.random.PRNGKey(FLAGS.train_init_random_seed)
        #rng = jnp.broadcast_to(rng, (local_device_count,) + rng.shape)
        out = dict(
            step=np.array(0),
            rng=out_rng,
            opt_state=opt_state,
            params=params,
            loss=np.array(0),
        )
        return out

    @functools.partial(jax.jit, static_argnums=0)
    def update(self, state: Mapping[str, Any], data: Mapping[str, jnp.ndarray]):
        """Updates the state using some data and returns metrics."""
        rng, new_rng = jax.random.split(state['rng'])
        params = state['params']
        loss, grads = jax.value_and_grad(self._loss_fn)(params, rng, data)

        if compute_on_tpu:
            grads = jax.lax.pmean(grads, 'i')


        updates, opt_state = self._opt.update(grads, state['opt_state'])
        params = optax.apply_updates(params, updates)

        new_state = {
            'step': state['step'] + 1,
            'rng': new_rng,
            'opt_state': opt_state,
            'params': params,
            'loss': state['loss'] + loss
        }

        return new_state 
    

In [ ]:
def mk_net_eval(net):
    def run_net(param, rng, data):
        return net.apply(param, rng, data, is_training=False)
    
    run_net = jax.pmap(run_net, axis_name='i')
    return run_net

def predict(net, state, data):
    predictions = []
    data = data.map(lambda x: ({'input_ids':x['input_ids']},
                               x['input_ids'],
                               x['text_id'], 
                               x['offset'])).as_numpy_iterator()
    
    
    for data, ids, text_id, offset  in data:    
        logits = net(state['params'], state['rng'], data)
        logits0 = jax.device_get(logits)
        del logits
        logits0 = logits0.reshape((-1,)+logits0.shape[2:])
        ## 0,1,2 are <s> </s> <pad>!
        mask = (ids>2).reshape((-1,)+ids.shape[2:])
        predictions.extend(zip(text_id.reshape(-1), 
                               offset.reshape(-1), 
                               (logit[m] for logit,m in zip(logits0, mask))))
                               #data['input_ids'].reshape((-1,)+data['input_ids']))
    return predictions

from collections import defaultdict
def collect_chunks(raw_predictions):
    predictions = defaultdict(list)
    for idx, offset, logit in raw_predictions:
        predictions[idx].append((offset, logit))
    return dict(predictions)

def combine_offset_list(offset_lst):
    offset_lst = [(offset, np.asarray(c)) for offset, c in offset_lst]
    total_len = max(offset+len(c) for offset, c in offset_lst)
    chunk_shape = offset_lst[0][1].shape
    res = np.zeros((total_len,)+chunk_shape[1:])
    cnt = np.zeros((total_len,)+(1,)*(len(chunk_shape)-1))
    for i, (o, c) in enumerate(offset_lst):
        res[o:o+len(c)] += c
        cnt[o:o+len(c)] += 1
    return res/cnt.clip(1)

from feedback2021.helper import label2id, id2label, Block
def pred_rle(pred):
    
    res = []
    cur_cls = -1
    min_cls = label2id.get('None',-1)
    
    for i, cls_id in enumerate(pred):
        
        if (cur_cls>min_cls) and (cur_cls!=cls_id):
            res.append(Block((cur_cls, start, i)))
            cur_cls = -1
        
        if cls_id>min_cls and cur_cls==-1: 
            cur_cls, start = cls_id, i
    
    if cur_cls>min_cls:
        res.append(Block((cur_cls, start, i+1)))
    
    return res


import  feedback2021.metric as metric

def compute_metric(net, state, chunked_data, raw_df):
    predictions = predict(net, state, chunked_data)
    predictions = collect_chunks(predictions)
    predictions = {k: pred_rle(np.argmax(combine_offset_list(v), axis=-1)) 
                   for k,v in predictions.items()}
    
    raw_data = zip(raw_df.index, raw_df['offset_mapping'], raw_df['word_mapping'])
    predictions = [
            [b.map(om).inv_map(wm) for b in predictions[idx]]
            for idx, om, wm in raw_data
        ]

    scores = np.array([ 
            metric.score_example(y_pred=y_pred, y_true=y_true)
            for y_pred,y_true in zip(predictions, raw_df['rle_word'])
        ]).sum(axis=0)
        
    res = {id2label[i+1]: metric.f1(*score) for i, score in enumerate(scores)}
    res['f1'] = sum(res.values())/len(res)
        
    return res
    
    
    

## predict(jax.pmap(net.apply, axis_name='i'), state, dataset)

        
def get_predictions(net, loss_fn, state, test_ds):

    def run_net(state, data):
        logits = net.apply(state['params'], state['rng'], data, is_training=False)
        loss = loss_fn(logits, data)
        pred = jax.numpy.argmax(logits, axis=-1)
        state['loss'] = state['loss'] + loss
        return pred, state
        
    
    run_net = jax.pmap(run_net, axis_name='i')
    predictions = []
    for batch in test_ds.as_numpy_iterator():
        pred, state = run_net(state, batch)
        predictions.append(pred)
        
    predictions = [ pred 
                    for preds in jax.device_get(predictions) 
                    for pred in preds.reshape((-1,) + preds.shape[2:]) ]
    return predictions, state

def eval_net(net, loss_fn, state, test_ds):

    def run_net(state, data, loss, cnt):
        cnt0 = jnp.greater(data['labels'].max(axis=-1),0).sum()
        logits = net.apply(state['params'], state['rng'], data, is_training=False)
        loss0 = loss_fn(logits, data) * data['labels'].shape[0]
        #pred = jax.numpy.argmax(logits, axis=-1)
        #state['loss'] = state['loss'] + loss
        return loss + loss0, cnt + cnt0
        
    
    run_net = jax.pmap(run_net, axis_name='i')
    #predictions = []
    loss = np.zeros(jax.device_count())
    cnt = np.zeros(jax.device_count())
    for batch in test_ds.as_numpy_iterator():
        loss, cnt = run_net(state, batch, loss, cnt)
        # predictions.append(pred)
    return float(loss.sum()/cnt.sum())

In [23]:
class Log:
    def __init__(self, step=0):
        self.step = step
        self.start_time = self.prev_time = time.time()
        
    def update(self, state):
        step = int(state['step'][0])
        step_delta, self.step = step-self.step, step
        
        c_time = time.time()
        time_delta, self.prev_time = c_time - self.prev_time, c_time
        
        loss = float(state['loss'].mean())/step_delta
        state['loss'] = 0*state['loss']
        
        return {'step': self.step,
                'loss': loss,
                'elapsed_time': time_delta,
                'total_time':   c_time-self.start_time,
                'iter_per_sec': time_delta/max(1,step_delta),
                'sec_per_iter': step_delta/max(1,time_delta),
               }

def format_log(log):
    return "iteration: {step:<5}, loss: {loss:.4f}, elapsed_time:{elapsed_time:.2f}".format(**log)

In [25]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint, #'distilroberta-base', 
                                    label2id=label2id, 
                                    id2label=id2label).to_dict()

config['decoder']= dict(
        num_token = max_rle_length,
        hidden_size = 64*12,
        initializer_range=0.02,
        hidden_dropout_prob = 0.1,
        attention_probs_dropout_prob=0.1,
        #layer_norm_eps = 1e-5,
        intermediate_size = 4*64*12,
        hidden_act = 'gelu',
        num_hidden_layers = 12, # 6,12,24,32?
        num_attention_heads = 12,
        )

config['final_attn'] = dict(
        key_size = 64, #16,#64*12,
        initializer_range=0.02,
        # hidden_dropout_prob = 0.1,
        # attention_probs_dropout_prob=0.1,
        # layer_norm_eps = 1e-5,
        # intermediate_size = 64, # 4*64*12
        # hidden_act = 'gelu',
        # num_hidden_layers = 3, # 6,12,24,32?
        num_attention_heads = 2,
        output_size = 2
    )
    
# config=dict(
#     decoder = dict(
#         num_token = 16,
#         hidden_size = 6*16,#64*12,
#         initializer_range=0.02,
#         hidden_dropout_prob = 0.1,
#         attention_probs_dropout_prob=0.1,
#         #layer_norm_eps = 1e-5,
#         intermediate_size = 64, # 4*64*12
#         hidden_act = 'gelu',
#         num_hidden_layers = 2, # 6,12,24,32?
#         num_attention_heads = 6,
#         ),
#     final_attn = dict(
#         key_size = 16,#64*12,
#         initializer_range=0.02,
#         # hidden_dropout_prob = 0.1,
#         # attention_probs_dropout_prob=0.1,
#         # layer_norm_eps = 1e-5,
#         # intermediate_size = 64, # 4*64*12
#         # hidden_act = 'gelu',
#         # num_hidden_layers = 3, # 6,12,24,32?
#         num_attention_heads = 2,
#         output_size = 2
#     ),
#     pad_token_id =1,
#     vocab_size = 10, #52000
#     max_position_embeddings = 32, # 514
    
#     hidden_size = 6*16,#64*12,
#     initializer_range=0.02,
#     hidden_dropout_prob = 0.1,
#     attention_probs_dropout_prob=0.1,
#     #layer_norm_eps = 1e-5,
#     intermediate_size = 64, # 4*64*12
#     hidden_act = 'gelu',
#     num_hidden_layers = 2, # 6,12,24,32?
#     num_attention_heads = 6,
#     id2label = id2label,
# )
config

{'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'use_bfloat16': False,
 'pruned_heads': {},
 'tie_word_embeddings': True,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'length_penalty': 1.0,
 'no_repeat_ngram_size': 0,
 'encoder_no_repeat_ngram_size': 0,
 'bad_words_ids': None,
 'num_return_sequences': 1,
 'chunk_size_feed_forward': 0,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': None,
 'forced_eos_token_id': None,
 'remove_invalid_values': False,
 'architectures': ['RobertaForMaskedLM'],
 'finetuning_task': None,
 'id2label': {0: 'None',
  1: 'Lead',
  2: 'Position',
  3: 'Evidence',
  4: 'Claim'

In [29]:
import logging
import time
from transformers import AutoConfig
from feedback2021.hk_roberta import (build_forward_fn, 
                                     lm_loss_fn, 
                                     # GradientUpdater, 
                                     hk, optax, functools, Mapping, jnp, jax)

## global variables
# batch_size = 16  # Train batch size per core
total_batch_size = batch_size*jax.device_count()
learning_rate = 1.25e-5*(total_batch_size/32) # Max learning-rate
grad_clip_value = 0.2  # Gradient norm clip value

checkpoint_dir = '/jax-transformer'  # Directory to store checkpoints
if just_test:
    LOG_EVERY = 2
    MAX_STEPS = 4
else:
    LOG_EVERY = len(ds['train'])//(2*total_batch_size)
    MAX_STEPS = (4*len(ds['train']))//total_batch_size+1

# config = AutoConfig.from_pretrained(model_checkpoint, #'distilroberta-base', 
#                                     label2id=label2id, 
#                                     id2label=id2label).to_dict()
translation = hk_roberta.weight_name_translation(config=config, prefix='roberta')
pretrained_params = hk_roberta.load_hf_pytorch_weights(model_checkpoint, #'distilroberta-base', 
                                                       translation)

logging.info = print
logging.info('Starting...')

backbone = hk.transform(mk_feature_fn(config))
net = hk.transform(mk_decoder_fn(config))

# forward_fn = build_forward_fn(config)
# forward_fn = hk.transform(forward_fn)

#loss_fn = functools.partial(lm_loss_fn, forward_fn.apply, config)
loss_fn = mk_loss_fn(config)

lr_schedule=optax.warmup_cosine_decay_schedule(init_value=0, 
                                               peak_value=1, 
                                               warmup_steps=min(50, MAX_STEPS//6), 
                                               decay_steps=MAX_STEPS-min(50,MAX_STEPS//6), 
                                               end_value=1e-4)

optimizer = optax.chain(
        optax.clip_by_global_norm(grad_clip_value),
        optax.adam(learning_rate, b1=0.9, b2=0.99),
        optax.scale_by_schedule(lr_schedule),
    )

updater = Updater2(backbone=backbone, 
                   net=net, 
                   loss_fn=loss_fn, 
                   optimizer=optimizer)

# Initialize parameters.
logging.info('Initializing parameters...')
rng = jax.random.PRNGKey(428)
batch = next(iter(train_dataset))
state = updater.init(rng, #jnp.broadcast_to(rng, (local_device_count,) + rng.shape), 
                     {k:v[0] for k,v in batch.items()},
                     pretrained_params=pretrained_params)
if compute_on_tpu:
    state = jax.device_put_replicated(state, jax.devices())
    update = jax.pmap(updater.update, axis_name='i')
else:
    update = updater.update
    


unused torch weights: roberta.embeddings.token_type_embeddings.weight,
roberta.pooler.dense.weight, roberta.pooler.dense.bias, lm_head.bias,
lm_head.dense.weight, lm_head.dense.bias, lm_head.layer_norm.weight,
lm_head.layer_norm.bias, lm_head.decoder.weight
Starting...
Initializing parameters...


In [ ]:
logging.info('Starting train loop...')
train_iter = train_dataset
log = Log()
net_eval_fn = mk_net_eval2(net, backbone)
for step  in range(MAX_STEPS): #enumerate(train_dataset):
        batch = next(train_iter)
        state = update(state, batch)
        # We use JAX runahead to mask data preprocessing and JAX dispatch overheads.
        # Using values from state/metrics too often will block the runahead and can
        # cause these overheads to become more prominent.
        if step % LOG_EVERY == 0 or (step+1)== MAX_STEPS:
            last_log=log.update(state)
            logging.info(last_log)
            print(compute_metric2(net_eval_fn, state, test_dataset, data['test']))
            # test_loss = eval_net(forward_fn, loss_fn, state, test_dataset)
            # last_log['test_loss'] = test_loss #float(state['loss'].mean())
            
            

# logging.info(last_log)

Starting train loop...
{'step': 1, 'loss': 47.71409225463867, 'elapsed_time': 94.47561693191528, 'total_time': 94.47561693191528, 'iter_per_sec': 94.47561693191528, 'sec_per_iter': 0.01058474167700497}


In [158]:
%%time
net_eval_fn = mk_net_eval2(net, backbone)
#raw_pred = predict2(net_eval, state, test_dataset)
#collect_chunks2(raw_pred)
print(compute_metric2(net_eval_fn, state, test_dataset, data['test']))
# data = test_dataset.map(lambda x: ({'input_ids':x['input_ids']},
#                                x['input_ids'],
#                                x['text_id'], 
#                                x['offset'])).as_numpy_iterator()
    
# params = state['params']
# backbone_params=state['backbone_params']
# rng = state['rng']
    
# for data, ids, text_id, offset  in data:   
#     # feature, mask = backbone.apply(backbone_params, rng1, data, is_training=False)
#     pos_logits, logits = net_eval(params, backbone_params, rng, data) 
#     break

{'Lead': 0.0, 'Position': 0.0, 'Evidence': 0.0, 'Claim': 0.0, 'Concluding Statement': 0.0, 'Counterclaim': 0.0, 'Rebuttal': 0.0, 'f1': 0.0}
CPU times: user 30.1 s, sys: 142 ms, total: 30.2 s
Wall time: 20.5 s


In [153]:
collect_chunks2(raw_pred)

{83: {(1, 190.0, 190.0),
  (4, 62.0, 63.0),
  (4, 190.0, 190.0),
  (4, 318.0, 319.0),
  (4, 446.0, 446.0),
  (4, 574.0, 575.0),
  (4, 902.0, 902.0)},
 53: {(1, 62.0, 63.0), (4, 62.0, 63.0)},
 70: {(4, 62.0, 62.0),
  (4, 190.0, 191.0),
  (4, 318.0, 318.0),
  (4, 446.0, 446.0),
  (4, 574.0, 574.0),
  (4, 623.0, 623.0)},
 45: {(4, 62.0, 63.0), (4, 190.0, 191.0)},
 44: {(4, 62.0, 62.0),
  (4, 190.0, 190.0),
  (5, 318.0, 319.0),
  (5, 362.0, 362.0)},
 39: {(4, 62.0, 63.0), (4, 190.0, 191.0), (4, 318.0, 318.0)},
 22: {(4, 62.0, 62.0),
  (4, 190.0, 190.0),
  (4, 318.0, 318.0),
  (4, 318.0, 319.0),
  (4, 385.0, 385.0),
  (4, 385.0, 386.0)},
 80: {(1, 62.0, 63.0)},
 10: set(),
 0: set(),
 -1: {(4, 62.0, 62.0)}}

In [145]:
(jax.nn.softmax(pos_logits.reshape(-1,*pos_logits.shape[2:]),axis=-3)[0,:,0,0]*jnp.arange(128)).sum()

DeviceArray(63.480347, dtype=float32)

In [106]:
jax.nn.one_hot(jnp.array([1,2]),3)

DeviceArray([[0., 1., 0.],
             [0., 0., 1.]], dtype=float32)

In [ ]:
net_eval_fn = mk_net_eval(forward_fn)

In [ ]:
%%time
print(compute_metric(net_eval_fn, state, test_dataset, data['test']))

In [ ]:
pred = get_predictions(forward_fn, loss_fn, state=state, test_ds=test_dataset)

In [ ]:
# next(test_dataset.as_numpy_iterator())
#next(iter(test_dataset))

In [ ]:
len(pred), pred[0].shape

In [ ]:
params = hk.data_structures.map(lambda name,module,x: x[0], state['params'])
import pickle
with open('params.pkl','wb') as f:
    pickle.dump(params, f)

In [ ]:
shraded_ds = [list(ds['train'].shard(8, i).as_numpy_iterator()) for i in range(8)]

In [ ]:
sharded_ds = jax.device_put_sharded(shards=shraded_ds, devices=jax.devices())

In [ ]:
lr_schedule=optax.warmup_cosine_decay_schedule(init_value=0, 
                                               peak_value=1, 
                                               warmup_steps=500, 
                                               decay_steps=8500, 
                                               end_value=1e-4)

In [ ]:
import jaxlib, jax
jaxlib.__version__, jax.__version__, hk.__version__

In [ ]:
cnt = np.arange(0,9000,10)
lr = [lr_schedule(jnp.array([i])) for i in cnt]
plt.plot(cnt, lr)

In [ ]:
{'loss': 2.4778389930725098, 'step': 0.0, 'steps_per_sec': 13.779059174634387}
{'loss': 1.3683964014053345, 'step': 500.0, 'steps_per_sec': 2.5767113856201855}
{'loss': 1.6631159782409668, 'step': 1000.0, 'steps_per_sec': 2.57725389686341}
{'loss': 1.3581798076629639, 'step': 1500.0, 'steps_per_sec': 2.5771785561488483}

In [ ]:
{k:v.shape for k,v in data.items()}

In [ ]:
jax.numpy.array([1,1])
#jax.device_get('gpu')
#jax.devices('gpu')
! nvcc --version

In [ ]:
from transformers import TrainingArguments
# TRAINING HYPERPARAMS
effective_batch_size = 32
max_steps = 4000
log_steps = 500
#n_step_examples = 8*500
batch_size = 16
grad_accumulation = effective_batch_size//batch_size
learning_rate = 5e-5
weight_decay = 0.01
warmup_ratio = 0.1
n_epochs = 3
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-{task}",
    evaluation_strategy = "steps",
    eval_steps = log_steps,
    logging_strategy = "steps",
    logging_steps = log_steps,
    save_strategy = "steps",
    save_steps = log_steps,
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    #num_train_epochs = n_epochs,
    max_steps = max_steps,
    weight_decay = weight_decay,
    report_to = 'wandb', 
    gradient_accumulation_steps = grad_accumulation,
    warmup_steps = int(1.5*(log_steps)),
    # logging_steps = 100,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
)
#training_args

In [ ]:
data['test']=add_rle_word2(data['test'])
data

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=chunk_data["train"].remove_columns(['offset', 'text_id']), #.select(range(100)),
    eval_dataset=chunk_data["test"].remove_columns(['offset', 'text_id']), #.select(range(100)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=mk_binary_metric(chunk_ds=chunk_data['test'],
                                     orig_ds=data['test'])
    #,
    #                                 min_word_cnt=[0,10,5,10,3,10,6,6]), #.select(range(100))), 
)

In [ ]:
trainer.train()
trainer.save_model(model_path)

In [ ]:
# wandb.init()
import pickle
from feedback2021.postprocess import mk_prediction_transform
for k,v in chunk_data.items():
    predictions = trainer.predict(v.remove_columns(['offset', 'text_id']))
    prediction_transform = mk_binary_prediction_transform(chunk_ds=v, orig_ds=data[k])
    predictions = prediction_transform(predictions[0])
    with open(f'{k}_predictions.pkl','wb') as f:
        pickle.dump(predictions, f)
    wandb.save(f'{k}_predictions.pkl','./','now')

In [ ]:
# ! rm longformer-base-4096-token_classification/ -rf
#!rm distilroberta-base-token_classification -rf
! ls -sh distilroberta-base-1

In [ ]:
wandb.finish()

# From this point on there is no need to gpu!

In [ ]:
prediction_file = wandb.restore('predictions.pkl', run_path='prvi/huggingface/d17yam8m')

In [ ]:
import os

In [ ]:
prediction_file_name = 'test_predictions.pkl' #if os.path.exists('test_predictions.pkl') else prediction_file.name
with open(prediction_file_name, 'rb') as f:
    saved_predictions = pickle.load(f)

In [ ]:
chunk_ds, orig_ds = chunk_data['test'].to_dict(), data['test'].to_dict()

In [ ]:
def filter_prediction(prediction, min_word_cnt):
    return [b for b in prediction if b[2]-b[1]>=min_word_cnt[b[0]]]

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
res = []
for mwc in tqdm(range(0,15)):
#     transform_prediction = mk_prediction_transform(chunk_ds=chunk_ds,
#                                                    orig_ds=orig_ds, min_word_cnt=mwc)
#     predictions = transform_prediction(saved_predictions[0])
    
    scores = np.array([metric.score_example(y_true=y_true, 
                                            y_pred=filter_prediction(y_pred,mwc)) 
              for y_pred,y_true in zip(saved_predictions,orig_ds['rle_word'])]).sum(axis=0)
    scores = [metric.f1(*score) for score in scores]
    f1 = sum(scores)/len(scores)
    scores = {id2label[i+1]:score for i,score in enumerate(scores)}
    scores['f1'] = f1
    scores['min_word_cnt']=mwc
    res.append(scores)

pd.DataFrame(res).set_index('min_word_cnt')

In [ ]:
mwc =[0, 10, 6, 15, 6, 11, 9, 8]

scores_ = np.array([metric.score_example(y_true=y_true, 
                                        y_pred=filter_prediction(y_pred,mwc)) 
              for y_pred,y_true in zip(saved_predictions,orig_ds['rle_word'])]).sum(axis=0)
scores = [metric.f1(*score) for score in scores_]
f1 = sum(scores)/len(scores)
scores = {id2label[i+1]:score for i,score in enumerate(scores)}
scores['f1'] = f1
    
# transform_prediction = mk_prediction_transform(chunk_ds=chunk_ds,
#                                                 orig_ds=orig_ds, 
#                                                min_word_cnt=[0, 10, 6, 15, 6, 11, 9, 8])
# predictions = transform_prediction(saved_predictions[0])
# scores = np.array([metric.score_example(y_true=y_true, y_pred=y_pred) 
#               for y_true,y_pred in zip(predictions,orig_ds['pred_range'])]).sum(axis=0)
# scores = [metric.f1(*score) for score in scores]
# f1 = sum(scores)/len(scores)
# scores = {id2label[i+1]:score for i,score in enumerate(scores)}
# scores['f1'] = f1
scores    

In [ ]:
scores_

In [ ]:
predictions = [ sorted(pred, key=lambda x: x[1]) for pred in saved_predictions]

In [ ]:
show_result(i=20, orig_ds=data['test'],predictions=predictions)

In [ ]:

saved_predictions[8],data['test'][8]['rle_token']

In [ ]:
scores = [metric.score_example(y_true=y_true, y_pred=y_pred) for y_true,y_pred in zip(predictions,orig_ds['pred_range'])]

In [ ]:
[metric.f1(*x) for x in np.array(scores).sum(axis=0)]

In [ ]:
idx = np.argsort(np.array(scores)[:,:,1:].sum(axis=(1,2)))[::-1]
idx[:10]